In [47]:
import pandas as pd
import os
import tqdm
filename='../FDDC/dingzeng/dingzeng.train'

In [48]:
def getDict(name,start=-1,end=-1,sentence=-1):
    return {'name':name,'start':start,'end':end,'sentence':sentence}
class hetong():
    def __init__(self,name,addObj,addType,addNum,addPrice,lookup,buyType):
        self.name=name
        self.addObj=[]
        self.addObj.append(getDict(addObj))
        self.addType=[]
        self.addType.append(getDict(addType))
        self.addNum=[]
        self.addNum.append(getDict(addNum))
        self.addPrice=[]
        self.addPrice.append(getDict(addPrice))
        self.lookup=[]
        self.lookup.append(getDict(lookup))
        self.buyType=[]
        self.buyType.append(getDict(buyType))
    

In [49]:
length,length2,ht=[],[],[]
with open(filename,'r') as fr:
    for line in fr.readlines():
        a=line.split('\t')
        length.append(len(a))
        if len(a)<7:
            a.extend(['']*(7-len(a)))
        length2.append(len(a))
        ht.append(hetong(*a))

In [50]:
dirname='../FDDC/dingzeng/textWithMoney'
if not os.path.exists(dirname):
    os.makedirs(dirname)
for i in tqdm.tqdm(ht):
    i=i.__dict__
    if i['addObj']=='' or i['addType']=='':
        continue
    name=i['name']
    i.pop('name')
    with open(os.path.join(dirname,name+'.txt'),'w') as fw:
        with open('../FDDC/dingzeng/dataHalf/'+name+'.txt','r') as fr:
            text=fr.readline()
            sentence=text.split('。')
            addObj_name=i['addObj'][0]['name']
            addType_name=i['addType'][0]['name']
            addNum_name=i['addNum'][0]['name']
            addPrice_name=i['addPrice'][0]['name']
            lookup_name=i['lookup'][0]['name']
            buyType_name=i['buyType'][0]['name']

            index=0
            for num,sen in enumerate(sentence):
                addObj_start=sen.find(addObj_name)
                addType_start=sen.find(addType_name)
                if addObj_start!=-1 and addType_start!=-1:
                    i['addObj'].append(getDict(addObj_name,index+addObj_start,index+addObj_start+len(addObj_name),num))
                    i['addType'].append(getDict(addType_name,index+addType_start,index+addType_start+len(addType_name),num))
                index+=len(sen)+1
            index=0
            for num,sen in enumerate(sentence):
                
                addObj_start=sen.find(addObj_name)
                addType_start=sen.find(addType_name)
                addNum_start=sen.find(addNum_name)
                addPrice_start=sen.find(addPrice_name)
                lookup_start=sen.find(lookup_name)
                buyType_start=sen.find(buyType_name)

                
                if addNum_name!='' and addNum_start !=-1 and (addObj_start!=-1 or addType_start!=-1):
                    i['addPrice'].append(getDict(addPrice_name,index+addPrice_start,index+addPrice_start+len(addPrice_name),num))
                    if addObj_start!=-1:
                        i['addObj'].append(getDict(addObj_name,index+addObj_start,index+addObj_start+len(addObj_name),num))
                    if addType_start!=-1:
                        i['addType'].append(getDict(addType_name,index+addType_start,index+addType_start+len(addType_name),num))   
                if addPrice_name!='' and addPrice_start !=-1 and (addObj_start!=-1 or addType_start!=-1):
                    i['addPrice'].append(getDict(addPrice_name,index+addPrice_start,index+addPrice_start+len(addPrice_name),num))
                    if addObj_start!=-1:
                        i['addObj'].append(getDict(addObj_name,index+addObj_start,index+addObj_start+len(addObj_name),num))
                    if addType_start!=-1:
                        i['addType'].append(getDict(addType_name,index+addType_start,index+addType_start+len(addType_name),num)) 
                if lookup_name!='' and lookup_start !=-1 and (addObj_start!=-1 or addType_start!=-1):
                    i['lookup'].append(getDict(lookup_name,index+lookup_start,index+lookup_start+len(lookup_name),num))
                    if addObj_start!=-1:
                        i['addObj'].append(getDict(addObj_name,index+addObj_start,index+addObj_start+len(addObj_name),num))
                    if addType_start!=-1:
                        i['addType'].append(getDict(addType_name,index+addType_start,index+addType_start+len(addType_name),num))   
                if buyType_name!='' and buyType_start !=-1 and (addObj_start!=-1 or addType_start!=-1):
                    i['buyType'].append(getDict(buyType_name,index+buyType_start,index+buyType_start+len(buyType_name),num))
                    if addObj_start!=-1:
                        i['addObj'].append(getDict(addObj_name,index+addObj_start,index+addObj_start+len(addObj_name),num))
                    if addType_start!=-1:
                        i['addType'].append(getDict(addType_name,index+addType_start,index+addType_start+len(addType_name),num))   
                index+=len(sen)+1

            li=['O' for i in text]

            for k,v in i.items():
                for item in v:
                    if item['name']!='' and item['start']!=-1:
                        li[item['start']]='B-'+k
                        for i in range(item['start']+1,item['end']):
                            li[i]='I-'+k
            for j,con in enumerate(li):
                if con!='O':
                    sub_start=j
                    break

            for j,con in enumerate(li):
                if li[len(li)-j-1]!='O':
                    sub_end=len(li)-j-1
                    break
            li=li[sub_start-10:sub_end+10]
            text=text[sub_start-10:sub_end+10]


            sss=""
            for j,con in enumerate(li):
                sss+=text[j]+' '+con+'\n'
            fw.write(sss)

                

100%|██████████| 13670/13670 [03:26<00:00, 66.35it/s] 


In [53]:
import os
import numpy as np
import random
dirname='../FDDC/dingzeng/textWithMoney'
files=list(os.walk(dirname))[0][2]
random.shuffle(files)
train="example.train"
test="example.test"
dev="example.dev"

with open(dev,'w') as fw:
    for file in files[int(len(files)*0.9):]:
        with open(os.path.join(dirname,file),'r') as fr:
            fw.writelines(fr.readlines())
        fw.write('\n')